# Computational Experiment 5: N-Body Simulations

In this experiment, you will run N-body simulations of a variety of systems to see how they evolve.

Before you begin, **make sure to read through the full notebook and understand what each function does**. Pay special attention to anything that has a **FIXME** note, which you will need to edit.

In this notebook, we will be making movies but saving them as external .mp4 files instead of embedding them in the notebook.

## 0. Import packages

In [ ]:
from gravhopper import Simulation, IC
from astropy import units as u, constants as const
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pynbody.analysis.profile as profile

import galpy.util
galpy.util.__config__.set('astropy', 'astropy-units', 'True')
from galpy import potential, df

## 1. Earth-Sun System

We will begin with a simple $N=2$ system so you can see how the package works: the Earth orbiting the Sun. First we need to set up the initial conditions (ICs) of the simulation: the Sun will be at the origin, and the Earth will be 1 au away on a circular orbit.

In [ ]:
# Sun at origin with zero velocity and mass 1 Msun
Sun = {'pos':np.array([0, 0, 0])*u.AU,
       'vel':np.array([0, 0, 0])*u.km/u.s,
       'mass':np.array([1])*u.Msun}
# Earth is at x=1 au, v_y=29.8 km/s and mass 1 Mearth
Earth = {'pos':np.array([1, 0, 0])*u.AU,
         'vel':np.array([0, 29.8, 0])*u.km/u.s,
         'mass':np.array([1])*u.Mearth}

To set up a simulation, we need to give two important parameters:
1. Time step $dt$, which defines how long between simulation steps.
2. The softening length $\epsilon$, which ensures that the force doesn't diverge when particles get too close. This is important when the particles represent samples of a smooth density field, as we'll get to later, but in this case each particle really represents a distinct object, so we don't want any softening and should set this to a very small number (for numerical reasons it's best if this isn't literally zero).

In [ ]:
# Set up a Simulation with a time step of 1/20th of a year, a very small softening
sim = Simulation(dt=0.05*u.yr, eps=0.001*u.au)

And then we add the two bodies to the ICs.

In [ ]:
sim.add_IC(Sun)
sim.add_IC(Earth)

Finally we run the simulation for 21 time steps. Each step is 1/20th of a year, so this will give us one full Earth orbit.

In [ ]:
sim.run(21)

After running, you can access the full position and velocity history using ``sim.positions`` and ``sim.velocities``. Each of these is a (Nsnap, Np, 3) array: the first dimension refers to the snapshot number, the second dimension refers to the particle number, and the third dimension refers to the coordinate (0=x, 1=y, 2=z). You can put specific numbers in for each of these, or use ``:`` to get them all.

For example, if you want to find the y velocities of all particles at snapshot number 5:
```
sim.velocities[5,:,1]
```
or if you wanted to know the 3D position of particle number 0 at the end (remember that you can use ``-1`` to get the last element):
```
sim.positions[-1,0,:]
```
or if you wanted to know the x positions of particle number 25 at all snapshots:
```
sim.positions[:,25,0]
```

So the following code will plot the x-y positions of each body over all snapshots.

In [ ]:
# Make a plot that has an aspect ratio of 1, i.e. it will be square if the axes have the same size.
plt.subplot(111, aspect=1.0)
# Plot x vs. y of particle 0 (Sun) in units of AU (default is kpc, which will look very strange!)
plt.plot(sim.positions[:,0,0].to(u.au), sim.positions[:,0,1].to(u.au), 'o', color='yellow')
# Plot x vs. y of particle 1 (Earth) as above
plt.plot(sim.positions[:,1,0].to(u.au), sim.positions[:,1,1].to(u.au), color='blue')
plt.xlabel('x (au)')
plt.ylabel('y (au)')

Look at it! Does it look like you'd expect?

Let's check out the Sun's velocity in one dimension, say $x$. You can access the time of each simulation snapshot using ``sim.times``.

In [ ]:
# Plot Sun's reflex motion vs. time (in years)
plt.plot(sim.times.to(u.yr), sim.velocities[:,0,0].to(u.m/u.s))
plt.xlabel('t (yr)')
plt.ylabel('$v_x$ (m/s)')

What do you think about this?

**FIXME:** Try changing the mass of the Earth. You will need to recreate the Simulation object each time -- there is a ``sim.reset()`` function, which will wipe out the run but it will leave the ICs and any external forces (see later), so if you want to completely change the ICs it's easiest to just create a new Simulation object by going back to the ``sim = Simulation(...`` line.

The first exoplanet-detecting spectrographs had a spectral resolution equivalent to 1 m/s. How massive does the Earth-like planet need to be to be detectable with those instruments around a nearby star, i.e. to cause a reflex motion of 1 m/s? Do you get the same answer from analytic arguments?

**FIXME**: Reset the mass of the Earth, but change the timestep ``dt`` and run the simulation for a few orbits (you will need to adjust the number of steps to be appropriate). What happens when ``dt`` is small? What happens when ``dt`` is large? How large can you make it before it starts messing up the orbit? Why?

## 2. Effects of orbital eccentricity

Let's try to make this a bit more realistic. You can use [this list of planetary data](https://nssdc.gsfc.nasa.gov/planetary/factsheet/) to look up the masses and orbital properties of some important Solar System bodies.

It's relatively easy to set up a planet at either aphelion or perihelion using this data: the "Orbital Velocity" and "Distance from Sun" rows are both averages, and conservation of angular momentum tells you that $v \propto 1/r$ throughout the orbit. For example, instead of the Earth shown above, this line will set up the Earth to be at perihelion along the x axis:

In [ ]:
#Make this the appropriate value in km/s but do not put the astropy unit in -- that will be done below.
v_Earth_at_peri = 29.8 * 149.6/147.1

Earth = {'pos':np.array([147.1e6, 0, 0])*u.km,
         'vel':np.array([0, v_Earth_at_peri, 0])*u.km/u.s,
         'mass':np.array([1])*u.Mearth}

Let's do the simulation again and see if accounting for orbital eccentricity makes a difference.

In [ ]:
# Set up a Simulation with a time step of 1/20th of a year, a very small softening
sim_e = Simulation(dt=0.05*u.yr, eps=0.001*u.au)
sim_e.add_IC(Sun)
sim_e.add_IC(Earth)
sim_e.run(21)

In [ ]:
# Make a plot that has an aspect ratio of 1, i.e. it will be square if the axes have the same size.
plt.subplot(111, aspect=1.0)
# Plot x vs. y of particle 0 (Sun) in units of AU
plt.plot(sim.positions[:,0,0].to(u.au), sim.positions[:,0,1].to(u.au), 'o', color='yellow')
# Plot x vs. y of circular Earth
plt.plot(sim.positions[:,1,0].to(u.au), sim.positions[:,1,1].to(u.au), color='blue', label='Circular')
# Plot x vs. y of eccentric Earth
plt.plot(sim_e.positions[:,1,0].to(u.au), sim_e.positions[:,1,1].to(u.au), color='darkcyan', label='Eccentric')
plt.xlabel('x (au)')
plt.ylabel('y (au)')
plt.legend()

In [ ]:
# Plot Sun's reflex motion vs. time (in years)
plt.plot(sim.times.to(u.yr), sim.velocities[:,0,0].to(u.m/u.s), color='blue', label='Circular')
plt.plot(sim_e.times.to(u.yr), sim_e.velocities[:,0,0].to(u.m/u.s), color='darkcyan', label='Eccentric')
plt.xlabel('t (yr)')
plt.ylabel('$v_x$ (m/s)')
plt.legend()

What do you think about these plots?

**FIXME:** Try changing the orbital eccentricity of the Earth, keeping everything else the same. How eccentric does it need to be to make a measurable difference? How do radial velocity curves for stars with eccentric planets differ?

## 3. Add more planets

**FIXME:** Using the orbital data from the NSSDC website, create a simulation that has Venus, Earth, Jupiter, and Saturn. Make sure to choose an appropriate value for both ``dt`` that is small enough to accurately integrate the fastest planet, and enough time steps to do at least one full orbit of the slowest planet. Set some of them at perihelion and some at aphelion. Also, don't line them all up on the same axis -- put at least a couple of them at different points in the orbit.

In [ ]:
Venus = #FIXME
Earth = #FIXME
Jupiter = #FIXME
Saturn = #FIXME

In [ ]:
sim_SS = Simulation(#FIXME
sim_SS.add_IC(#FIXME
#...more add_IC lines...
sim_SS.run(#FIXME

Plot the planet orbits.

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, aspect=1.0)
ax.plot(sim_SS.positions[:,0,0].to(u.au), sim_SS.positions[:,0,1].to(u.au), 'o', color='yellow')
for i in range(1,5):
    ax.plot(sim_SS.positions[:,i,0].to(u.au), sim_SS.positions[:,i,1].to(u.au))

What do you think?

We can make a movie of the simulation using the ``movie_particles()`` function. It will write the output to an .mp4 file (you need to give it a filename). The other most useful parameters are the unit of the x and y axes, and the x and y limits of the plots that make up the movie. If you want to do a more complicated movie, you can follow the examples later on or look at the GravHopper source code of this function.

In [ ]:
# The title bar will show a timeline, but by default it is in Myr, which isn't very useful for this case.
# So first let's change the sim_SS.times array to be in units of years.
sim_SS.times = sim_SS.times.to(u.yr)
# Then make the movie, with axes in units of AU, going out to 12 AU in each direction, and points that are not too small.
sim_SS.movie_particles('SS.mp4', unit=u.au, xlim=[-12,12], ylim=[-12,12], s=8)

What do you think of this movie?

Now let's look at the Solar reflex motion again.

In [ ]:
# Plot Sun's reflex motion vs. time (in years)
plt.plot(sim_SS.times.to(u.yr), sim_SS.velocities[:,0,0].to(u.m/u.s))
plt.xlabel('t (yr)')
plt.ylabel('$v_x$ (m/s)')

**FIXME:** Use ``plt.xlim()`` and ``plt.ylim`` to zoom in on diffent parts of the plot (or you can run this all with ``%matplotlib notebook`` so you can zoom in interactively). Can you identify oscillations caused by each of the planets?

## 4. Do something interesting

Alter one of the above simulations in some interesting way to explore solar system-type dynamics. For example, you could:
 - Add one or more moons.
 - Make planets massive enough and/or nearby enough to each other that they interact with each other.
 - Throw a planet at the Sun.
 - Simulate a real exoplanet system.
 - Set up an unstable 3-body system.